In [3]:
import copy
import keras
import numpy as np
from model import get_model
from scripts.data_preprocess import get_data

2023-03-29 17:30:20.115468: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 17:30:20.200005: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
x_train, x_test, y_train, y_test = get_data('sudoku.csv')

In [5]:
model = get_model()

adam = keras.optimizers.Adam(lr=.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

model.fit(x_train, y_train, batch_size=32, epochs=2)

2023-03-29 16:22:34.972027: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/nix/store/jzi98h678vkjd1zi9c9yi7f4q91vzxkj-python3-3.10.9-env/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/2
25000/25000 [==============================] - 833s 33ms/step - loss: 0.4469
Epoch 2/2
25000/25000 [==============================] - 832s 33ms/step - loss: 0.3601


In [4]:
#model.save('model/sudoku.model')
model = keras.models.load_model('model/sudoku.model')

2023-03-29 17:30:29.750400: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def norm(a):    
    return (a/9)-.5

In [6]:
def denorm(a):
    return (a+.5)*9

In [7]:
def inference_sudoku(sample):
    
    '''
        This function solve the sudoku by filling blank positions one by one.
    '''
    
    feat = copy.copy(sample)
    
    while(1):
    
        out = model.predict(feat.reshape((1,9,9,1)))  
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1 
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
     
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
    
        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    
    return pred

In [8]:
def test_accuracy(feats, labels):
    
    correct = 0
    
    for i,feat in enumerate(feats):
        
        pred = inference_sudoku(feat)
        
        true = labels[i].reshape((9,9))+1
        
        if(abs(true - pred).sum()==0):
            correct += 1
        
    print(correct/feats.shape[0])

In [13]:
test_accuracy(x_test[:100], y_test[:100])

1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step


...

1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 18ms/step
1.0


In [9]:
def solve_sudoku(game):
    
    game = game.replace('\n', '')
    print(game)
    game = game.replace(' ', '')
    print(game)
    game = np.array([int(j) for j in game]).reshape((9,9,1))
    print(game)
    game = norm(game)
    print(game)
    game = inference_sudoku(game)
    print(game)
    return game

In [10]:
game = '''
          0 8 0 0 3 2 0 0 1
          7 0 3 0 8 0 0 0 2
          5 0 0 0 0 7 0 3 0
          0 5 0 0 0 1 9 7 0
          6 0 0 7 0 9 0 0 8
          0 4 7 2 0 0 0 5 0
          0 2 0 6 0 0 0 0 9
          8 0 0 0 9 0 3 0 5
          3 0 0 8 2 0 0 1 0
      '''

game = solve_sudoku(game)

print('solved puzzle:\n')
print(game)

          0 8 0 0 3 2 0 0 1          7 0 3 0 8 0 0 0 2          5 0 0 0 0 7 0 3 0          0 5 0 0 0 1 9 7 0          6 0 0 7 0 9 0 0 8          0 4 7 2 0 0 0 5 0          0 2 0 6 0 0 0 0 9          8 0 0 0 9 0 3 0 5          3 0 0 8 2 0 0 1 0      
080032001703080002500007030050001970600709008047200050020600009800090305300820010
[[[0]
  [8]
  [0]
  [0]
  [3]
  [2]
  [0]
  [0]
  [1]]

 [[7]
  [0]
  [3]
  [0]
  [8]
  [0]
  [0]
  [0]
  [2]]

 [[5]
  [0]
  [0]
  [0]
  [0]
  [7]
  [0]
  [3]
  [0]]

 [[0]
  [5]
  [0]
  [0]
  [0]
  [1]
  [9]
  [7]
  [0]]

 [[6]
  [0]
  [0]
  [7]
  [0]
  [9]
  [0]
  [0]
  [8]]

 [[0]
  [4]
  [7]
  [2]
  [0]
  [0]
  [0]
  [5]
  [0]]

 [[0]
  [2]
  [0]
  [6]
  [0]
  [0]
  [0]
  [0]
  [9]]

 [[8]
  [0]
  [0]
  [0]
  [9]
  [0]
  [3]
  [0]
  [5]]

 [[3]
  [0]
  [0]
  [8]
  [2]
  [0]
  [0]
  [1]
  [0]]]
[[[-0.5       ]
  [ 0.38888889]
  [-0.5       ]
  [-0.5       ]
  [-0.16666667]
  [-0.27777778]
  [-0.5       ]
  [-0.5       ]
  [-0.38888889]]

 [[ 0.27777778]
  

In [17]:
np.sum(game, axis=1)

array([45, 45, 45, 45, 45, 45, 45, 45, 45])

In [20]:
model2 = keras.models.load_model('model/sudoku.model')
model2